In [19]:
import pandas as pd
import numpy as np
from io import StringIO
import rasterio

In [2]:
df_urls = pd.read_csv('../resources/mp16_urls.csv', names=['IMG_ID', 'URL'])
df_filtered = pd.read_csv('../resources/MP16_Pro_filtered.csv')
df_places = pd.read_csv('../resources/mp16_places365.csv')

In [10]:
df_urls.head()

,IMG_ID,URL
0,92/17/5276763594.jpg,http://farm6.staticflickr.com/5042/5276763594_...
1,0d/ce/6392770405.jpg,http://farm8.staticflickr.com/7172/6392770405_...
2,2a/88/5268406683.jpg,http://farm6.staticflickr.com/5045/5268406683_...
3,82/be/2515710583.jpg,http://farm3.staticflickr.com/2389/2515710583_...
4,03/05/9498368699.jpg,http://farm4.staticflickr.com/3800/9498368699_...


In [11]:
df_filtered.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban,neighbourhood,city,county,state,region,country,country_code,continent
0,92_17_5276763594.jpg,42441750@N03,38.685568,-109.532951,1.0,7.0,289.0,1.739840e-04,0.897409,0.102417,NaN,NaN,Grand County,Utah,NaN,United States,us,NaN
1,0d_ce_6392770405.jpg,68149505@N00,34.933793,103.692741,0.0,1.0,122.0,9.968868e-01,0.000578,0.002535,NaN,Lianlu,Kangle County,Gansu,Linxia,China,cn,NaN
2,2a_88_5268406683.jpg,84867026@N00,39.983433,-75.243301,0.0,0.0,128.0,7.201538e-01,0.034871,0.244975,Overbrook,Philadelphia,Philadelphia County,Pennsylvania,NaN,United States,us,NaN
3,82_be_2515710583.jpg,75292316@N00,39.306094,-84.379291,1.0,6.0,145.0,9.050690e-05,0.516982,0.482927,NaN,NaN,Butler County,Ohio,NaN,United States,us,NaN
4,03_05_9498368699.jpg,61068860@N00,9.186625,123.581597,1.0,8.0,36.0,9.902391e-07,0.999983,0.000016,NaN,Siquijor,NaN,Siquijor,Central Visayas,Philippines,ph,NaN


In [12]:
df_places.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban
0,92/17/5276763594.jpg,42441750@N03,38.685568,-109.532951,1,7,289,1.739840e-04,0.897409,0.102417
1,0d/ce/6392770405.jpg,68149505@N00,34.933793,103.692741,0,1,122,9.968868e-01,0.000578,0.002535
2,2a/88/5268406683.jpg,84867026@N00,39.983433,-75.243301,0,0,128,7.201538e-01,0.034871,0.244975
3,82/be/2515710583.jpg,75292316@N00,39.306094,-84.379291,1,6,145,9.050690e-05,0.516982,0.482927
4,03/05/9498368699.jpg,61068860@N00,9.186625,123.581597,1,8,36,9.902391e-07,0.999983,0.000016


In [13]:
df_urls.rename(columns={'IMG_ID_PATH': 'IMG_ID'}, inplace=True)
df_filtered.rename(columns={'IMG_ID_UNDERSCORE': 'IMG_ID'}, inplace=True)

df_urls['Merge_Key'] = df_urls['IMG_ID'].str.replace('/', '_')

df_combined = pd.merge(
    df_filtered, 
    df_urls[['URL', 'Merge_Key']], 
    left_on='IMG_ID', 
    right_on='Merge_Key', 
    how='inner'
)

df_combined.drop(columns=['Merge_Key'], inplace=True)

In [15]:
df_combined.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban,neighbourhood,city,county,state,region,country,country_code,continent,URL
0,92_17_5276763594.jpg,42441750@N03,38.685568,-109.532951,1.0,7.0,289.0,1.739840e-04,0.897409,0.102417,NaN,NaN,Grand County,Utah,NaN,United States,us,NaN,http://farm6.staticflickr.com/5042/5276763594_...
1,0d_ce_6392770405.jpg,68149505@N00,34.933793,103.692741,0.0,1.0,122.0,9.968868e-01,0.000578,0.002535,NaN,Lianlu,Kangle County,Gansu,Linxia,China,cn,NaN,http://farm8.staticflickr.com/7172/6392770405_...
2,2a_88_5268406683.jpg,84867026@N00,39.983433,-75.243301,0.0,0.0,128.0,7.201538e-01,0.034871,0.244975,Overbrook,Philadelphia,Philadelphia County,Pennsylvania,NaN,United States,us,NaN,http://farm6.staticflickr.com/5045/5268406683_...
3,82_be_2515710583.jpg,75292316@N00,39.306094,-84.379291,1.0,6.0,145.0,9.050690e-05,0.516982,0.482927,NaN,NaN,Butler County,Ohio,NaN,United States,us,NaN,http://farm3.staticflickr.com/2389/2515710583_...
4,03_05_9498368699.jpg,61068860@N00,9.186625,123.581597,1.0,8.0,36.0,9.902391e-07,0.999983,0.000016,NaN,Siquijor,NaN,Siquijor,Central Visayas,Philippines,ph,NaN,http://farm4.staticflickr.com/3800/9498368699_...


In [45]:
df_kg = pd.read_csv('../resources/kg_leg.csv', quotechar='"', skipinitialspace=True)
print(df_kg["Climate"][1])

Tropical, monsoon


In [55]:
def get_climate(lat, lon):
    with rasterio.open("../resources/koppen_geiger.tif") as src:
        row, col = src.index(lon, lat)
        value = src.read(1)[row, col]
        if value > 31:
            return "other"
        climate = df_kg["Climate"][value]
        return climate

In [ ]:
def create_geo_caption(row):
    caption = "A geo-tagged image"
    location_parts = []
    
    if pd.notna(row['city']):
        location_parts.append(row['city'])
    if pd.notna(row['state']):
        location_parts.append(row['state'])
    if pd.notna(row['country']):
        location_parts.append(row['country'])

    if location_parts:
        caption += f" taken in {', '.join(location_parts)}."
    else:
        caption += "."

    if (row['Prob_indoor'] > 0.8):
        caption += " This image was taken indoors."
    if (row['Prob_natural'] > 0.8):
        caption += " This image was taken in nature."
    if (row['Prob_indoor'] > 0.8):
        caption += " This image was taken in a urban setting."
    
    if pd.notna(row['LAT']) and pd.notna(row['LON']):
        caption += f"The climate is {get_climate(row['LAT'], row['LON'])}."
    
        
    return caption

df_combined['caption'] = df_combined.apply(create_geo_caption, axis=1)

KeyError: np.uint8(30)

In [ ]:
df_combined.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban,neighbourhood,city,county,state,region,country,country_code,continent,URL,caption
0,92_17_5276763594.jpg,42441750@N03,38.685568,-109.532951,1.0,7.0,289.0,1.739840e-04,0.897409,0.102417,NaN,NaN,Grand County,Utah,NaN,United States,us,NaN,http://farm6.staticflickr.com/5042/5276763594_...,"A geo-tagged image taken in Utah, United States."
1,0d_ce_6392770405.jpg,68149505@N00,34.933793,103.692741,0.0,1.0,122.0,9.968868e-01,0.000578,0.002535,NaN,Lianlu,Kangle County,Gansu,Linxia,China,cn,NaN,http://farm8.staticflickr.com/7172/6392770405_...,"A geo-tagged image taken in Lianlu, Gansu, China."
2,2a_88_5268406683.jpg,84867026@N00,39.983433,-75.243301,0.0,0.0,128.0,7.201538e-01,0.034871,0.244975,Overbrook,Philadelphia,Philadelphia County,Pennsylvania,NaN,United States,us,NaN,http://farm6.staticflickr.com/5045/5268406683_...,"A geo-tagged image taken in Philadelphia, Penn..."
3,82_be_2515710583.jpg,75292316@N00,39.306094,-84.379291,1.0,6.0,145.0,9.050690e-05,0.516982,0.482927,NaN,NaN,Butler County,Ohio,NaN,United States,us,NaN,http://farm3.staticflickr.com/2389/2515710583_...,"A geo-tagged image taken in Ohio, United States."
4,03_05_9498368699.jpg,61068860@N00,9.186625,123.581597,1.0,8.0,36.0,9.902391e-07,0.999983,0.000016,NaN,Siquijor,NaN,Siquijor,Central Visayas,Philippines,ph,NaN,http://farm4.staticflickr.com/3800/9498368699_...,"A geo-tagged image taken in Siquijor, Siquijor..."


In [18]:
df_combined.to_csv('../resources/mp16_combined.csv', index=False)